In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf
import datetime as date
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
yf.pdr_override()
df = yf.download("SBIN.NS", start='2015-01-01', end=date.datetime.now())


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
#removed date column
df = df.drop(['Date', 'Adj Close'], axis= 1)
df.head()

In [ ]:
plt.plot(df.Close)



In [ ]:
df

In [ ]:
#moving avg indicator(ma100)
ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
#plotting of ma100
plt.figure(figsize= (12,6))
plt.plot(df.Close)
plt.plot(ma100,'r')


In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200



In [ ]:
plt.figure(figsize= (12,6))
plt.plot(df.Close)
plt.plot(ma100,'r')
plt.plot(ma200,'g')

In [ ]:
#showing data that how many row and columns we have 
df.shape

In [ ]:
#Splitting Data into Training and Tensting(70%data for training & 30% data is for testing)

data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70): int(len(df))])

print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training

In [36]:
len(data_testing)


648

In [ ]:
#for stck lstm model we can't provide "Close(or any data)" so i give (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))


In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
data_training_array.shape

In [ ]:
#creating list for coordinate train
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i, 0])

x_train,  y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
#MACHINE LEARNIG MODEL...# last (x_train.shape[1], 1)<--(this 1 is for 'close' column can be different for diffrent data)
model = Sequential()
model.add(LSTM(units= 50, activation='relu', return_sequences= True,
               input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units= 60, activation='relu', return_sequences= True,))
model.add(Dropout(0.3))

model.add(LSTM(units= 80, activation='relu', return_sequences= True,))
model.add(Dropout(0.4))

model.add(LSTM(units= 120, activation='relu'))
model.add(Dropout(0.5))

#dense for connect all upper layers
model.add(Dense(units= 1))


In [ ]:
model.summary()

In [ ]:
''' 
learn optimizers(adam must)
      what is epochs
'''

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs = 50)


In [ ]:
#model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
data_training.tail(100)

In [ ]:
data_training.tail(100)


In [ ]:
past_100_days = data_training.tail(100)
final_df = pd.concat([past_100_days, data_testing], ignore_index=True)


In [ ]:
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100: i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# making Prediction

y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.00337098
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test, 'b', label = 'Original Price')
plt.plot(y_predicted, 'r', label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()
